<a href="https://colab.research.google.com/github/llA-Wilsonll/Simpsons_Character_Recognition/blob/master/The_Simpsons_Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
In the previous notebook, I built (simple) models from scratch to try and classify images of 14 differnt simpsons characters. Due to the small dataset and the resources I have available to me to train the model, it wasn't feasible to train a super complex model to classify these images.

Thankfully, there are many publicly available models which have been trained on much more data for much longer than we ever could. We can piggy-back off these models using their weights to extract features from our own images. To do so we will download a model and freeze it's layers up to a certain point, after which we will add some of our own dense layer which we will train.

# Set up
Loading the required packages

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

In [0]:
import re
import itertools
import numpy as np
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import matplotlib.image  as mpimg
import matplotlib.pyplot as plt
import matplotlib.axes as ax
from sklearn.metrics import classification_report, confusion_matrix

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile 
import os
import random

Loading our dataset we saved to Google Drive in an earlier notebook

In [0]:
# Insert your file ID below
# An example : https://drive.google.com/file/d/1iz5JmTB4YcBvO7amj3Sy2_scSeAsN4gd/view?usp=sharing
zip_id = '1HGcEPmGBYAXIp4TUdiIlYodBdHZswNYB'

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

if not os.path.exists('/content/simpsons_generator_v2'):
    os.makedirs('/content/simpsons_generator_v2')

# 2. Download Zip
print ("Downloading zip file")
myzip = drive.CreateFile({'id': zip_id})
myzip.GetContentFile('simpsons_generator_directory_v2.zip')

# 3. Unzip
print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('simpsons_generator_directory_v2.zip', 'r')
zip_ref.extractall('/content/simpsons_generator_v2/')
zip_ref.close()